In [2]:
!pip install telethon


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 11.3 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=45e3541f0bb5b22a8f77ea22de2d3f62fd59d73a121034637a19f4f5740807f1
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [3]:
import logging
import csv
import os
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel


In [4]:
api_id = ''
api_hash = ''
phone_number = ''
client = TelegramClient('session_name', api_id, api_hash)

await client.start(phone_number)


Please enter the code you received: 41748
Signed in successfully as Yenat; remember to not break the ToS or you will risk an account ban!


In [6]:
async def scrape_channel(channel_username):
    await client.connect()
    if not await client.is_user_authorized():
        await client.send_code_request(phone_number)
        await client.sign_in(phone_number, input('Enter the code: '))

    entity = await client.get_entity(channel_username)
    my_channel = PeerChannel(entity.id)
    offset_id = 0
    limit = 100
    all_messages = []

    while True:
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))

        if not history.messages:
            break

        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())

        offset_id = messages[len(messages) - 1].id

    await client.disconnect()
    return all_messages

# Replace 'CheMed123' with the correct channel username
messages = await scrape_channel('CheMed123')

output_file = 'chemed_channel_data.csv'
with open(output_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'date', 'message'])
    for message in messages:
        writer.writerow([message['id'], message['date'], message.get('message', '')])

logging.basicConfig(filename='scraping.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Started scraping CheMed123 Telegram Channel')

try:
    messages = await scrape_channel('CheMed123')
    logging.info('Scraping completed successfully')
except Exception as e:
    logging.error(f'Error during scraping: {e}')

In [7]:
output_file = 'chemed_channel_data.csv'
with open(output_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'date', 'message'])
    for message in messages:
        writer.writerow([message['id'], message['date'], message.get('message', '')])


In [10]:
logging.basicConfig(filename='scraping.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Started scraping Chemed Telegram Channel')

try:
    messages = await scrape_channel('CheMed123')
    logging.info('Scraping completed successfully')
except Exception as e:
    logging.error(f'Error during scraping: {e}')


In [ ]:
import asyncio

async def scrape_images(channel_username, output_dir):
    await client.connect()
    if not await client.is_user_authorized():
        await client.send_code_request(phone_number)
        await client.sign_in(phone_number, input('Enter the code: '))

    entity = await client.get_entity(channel_username)
    my_channel = PeerChannel(entity.id)
    offset_id = 0
    limit = 100

    while True:
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))

        if not history.messages:
            break

        messages = history.messages
        for message in messages:
            if message.media:
                try:
                    await client.download_media(message, output_dir)
                except Exception as e:
                    logging.error(f"Failed to download media: {e}")

        offset_id = messages[len(messages) - 1].id

    await client.disconnect()

# Set output directories
os.makedirs('images/chemed', exist_ok=True)
os.makedirs('images/lobelia', exist_ok=True)

# Scrape images from both channels
await scrape_images('CheMed123', 'images/chemed')
await scrape_images('lobelia4cosmetics', 'images/lobelia')


In [ ]:
logging.basicConfig(filename='image_scraping.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Started scraping images from CheMed123 and lobelia4cosmetics Telegram Channels')

try:
    await scrape_images('CheMed123', 'images/chemed')
    await scrape_images('lobelia4cosmetics', 'images/lobelia')
    logging.info('Image scraping completed successfully')
except Exception as e:
    logging.error(f'Error during image scraping: {e}')
